**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [32]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [33]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [34]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act refers to the conditions on the action to be performed by the agent, under the given state of environment. Sometimes, there may be multiple ways to act at a particular state but, we can make the agent choose only one action. Normally, the agent chooses the action which is much into the future no matter how good the other options are. To overcome this partiality, we use the term exploration.

The way of representing exploration is through an additional term in the equation which is __Epsilon__. If the randomly generated value is less tha Epsilon, then the random action is chosen, else we will choose the path with the maximum Q value

$$ a^{*} = argmax_{a} Q^{\pi} (s , a) $$

In short, Epsilon basically represents the tradeoff between exploration and exploitation, which allows our agent to either explore more options or to exploit the most common option to the maximum

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [35]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [36]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is the representation of the position of the rat on the board at a given time and state. The array is updated for each call of the function __act__  by setting all other boxes except the position of the rat to zeros. The goal of the array is to keep track of the position of the mouse and give information about its path to the agent. 

```board``` is the array that consists of all the possible positions on which the rat can move around. Its elements correspond to the rewards (-1 for poison and 0.5 for cheese) and 0 if there is it is plain space. It is useful for defining the state of the environment. The distribution of the reward (0.5 and -1) follows a binomial law. 

The environment sends a partial view of ```position``` and  ```board``` (visibility of 2 cells from its current position : self.x and self.y) to the rat so that it can move around based on the available information

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [37]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return random.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [28]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
                
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [38]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.0/13.0. Average score (-7.0)
Win/lose count 15.5/12.0. Average score (-1.75)
Win/lose count 6.0/17.0. Average score (-4.833333333333333)
Win/lose count 10.5/7.0. Average score (-2.75)
Win/lose count 6.0/13.0. Average score (-3.6)
Win/lose count 10.0/13.0. Average score (-3.5)
Win/lose count 7.0/18.0. Average score (-4.571428571428571)
Win/lose count 9.0/15.0. Average score (-4.75)
Win/lose count 9.0/11.0. Average score (-4.444444444444445)
Win/lose count 10.0/13.0. Average score (-4.3)
Win/lose count 10.0/26.0. Average score (-5.363636363636363)
Win/lose count 14.0/12.0. Average score (-4.75)
Win/lose count 8.5/13.0. Average score (-4.730769230769231)
Win/lose count 11.5/11.0. Average score (-4.357142857142857)
Win/lose count 11.5/20.0. Average score (-4.633333333333334)
Win/lose count 9.5/20.0. Average score (-5.0)
Win/lose count 10.0/19.0. Average score (-5.235294117647059)
Win/lose count 9.5/18.0. Average score (-5.416666666666667)
Win/lose count 9.0/15.0. Average s

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### Answer:
According to the definition:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

So:


\begin{align}Q^\pi(s,a)&=E_{p^{\pi}}[r(s_{0},a_{0}) + \sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0} =s,a_{0}=a] 
\\
&=E_{p^{\pi}}[r(s_{0},a_{0}) + \gamma\sum_{t=1}^{+\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0} =s,a_{0}=a]
\\
&=E_{p^{\pi}}[r(s_{0},a_{0})|s_{0} =s,a_{0}=a] + E_{p^{\pi}}[\gamma\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0} =s,a_{0}=a]
\\
&=r(s,a) + \gamma E_{p^{\pi}(.|s_{0} =s,a_{0}=a)}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})]
\\
&=r(s,a) + \gamma \sum_{s',a'}p^{\pi}(s_{1}=s',a_{1}=a'|s_{0} =s,a_{0}=a)E_{p^{\pi}(.|s_{0} =s,a_{0}=a)}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1} =s',a_{1}=a']
\\
&=r(s,a) + \gamma \sum_{s',a'}p^{\pi}(s_{1}=s',a_{1}=a'|s_{0} =s,a_{0}=a)E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0} =s',a_{0}=a']
\\
&=r(s,a) + \gamma \sum_{s',a'}p^{\pi}(s_{1}=s',a_{1}=a'|s_{0} =s,a_{0}=a)Q^{\pi}(s',a')
\\
Q^\pi(s,a)&=E_{(s′,a′)∼p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]
\end{align}

 

Assuming the optimal policy $\pi^*$ exists, the optimal Q function is defined as follows:

\begin{align}Q^*(s,a)&=\max_{\pi}Q^\pi(s,a)
\\
&= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\\
&= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')]
\\
Q^*(s,a)&= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align}

Finally, the minimization of the loss function is given by:

\begin{align}\mathcal{L}(\theta)&= E_{s' \sim \pi^*(.|s,a)}( ( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2) 
\\
&=  E_{s' \sim \pi^*(.|s,a)} ( ( r(s,a)+\gamma\max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2) 
\\
&= E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{align}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [40]:
#from collections import deque
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory: # if memory is not full
            self.memory.append(m)
        else:
            self.memory[np.random.randint(0, len(self.memory), size=1)[0]] = m

    def random_access(self):
        idx = random.randint(0,len(self.memory)-1)
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [41]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [42]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        
        s = np.reshape(s,[1,5,5,self.n_state])
        # Predict the reward value based on the given state
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            target = r_
            minibatch = self.memory.random_access()
            
            s_= minibatch[0]
            n_s_= minibatch[1]
            a_ = minibatch[2]
            r_= minibatch[3]
            game_over_= minibatch[4]
            
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                
            target_f = self.model.predict(s_)
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            

class DQN_FC(DQN):
    def __init__(self, lr=0.1,  *args, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(512, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model
        
        

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 1.0828 | Win/lose count 3.0/6.0 (-3.0)
Epoch 001/020 | Loss 0.1244 | Win/lose count 5.0/8.0 (-3.0)
Epoch 002/020 | Loss 0.3165 | Win/lose count 13.0/20.0 (-7.0)
Epoch 003/020 | Loss 0.1395 | Win/lose count 4.5/9.0 (-4.5)
Epoch 004/020 | Loss 0.9537 | Win/lose count 7.5/11.0 (-3.5)
Epoch 005/020 | Loss 0.2668 | Win/lose count 7.5/12.0 (-4.5)
Epoch 006/020 | Loss 0.2534 | Win/lose count 8.0/8.0 (0.0)
Epoch 007/020 | Loss 0.2024 | Win/lose count 4.0/12.0 (-8.0)
Epoch 008/020 | Loss 0.4752 | Win/lose count 7.0/12.0 (-5.0)
Epoch 009/020 | Loss 0.5957 | Win/lose count 12.0/4.0 (8.0)
Epoch 010/020 | Loss 0.2777 | Win/lose count 10.5/17.0 (-6.5)
Epoch 011/020 | Loss 1.1697 | Win/lose count 7.0/23.0 (-16.0)
Epoch 012/020 | Loss 0.3500 | Win/lose count 5.5/10.0 (-4.5)
Epoch 013/020 | Loss 0.3869 | Win/lose count 12.0/11.0 (1.0)
Epoch 014/020 | Loss 0.8928 | Win/lose count 9.0/13.0 (-4.0)
Epoch 015/020 | Loss 0.5696 | Win/lose count 7.5/11.0 (-3.5)
Epoch 016/020 | Loss 0.3364

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [46]:
from keras.layers.core import Flatten
from keras.layers.core import Dropout

class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 32
        kernel_size = (2,2)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
       

In [47]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/020 | Loss nan | Win/lose count 3.0/5.0 (-2.0)
Epoch 001/020 | Loss nan | Win/lose count 2.0/5.0 (-3.0)
Epoch 002/020 | Loss nan | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/020 | Loss nan | Win/lose count 3.0/5.0 (-2.0)
Epoch 004/020 | Loss nan | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/020 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 006/020 | Loss nan | Win/lose count 1.5/2.0 (-0.5)
Epoch 007/020 | Loss nan | Win/lose count 0.5/2.0 (-1.5)
Epoch 008/020 | Loss nan | Win/lose count 1.5/2.0 (-0.5)
Epoch 009/020 | Loss nan | Win/lose count 1.0/4.0 (-3.0)
Epoch 010/020 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 011/020 | Loss nan | Win/lose count 2.5/1.0 (1.5)
Epoch 012/020 | Loss nan | Win/lose count 2.0/1.0 (1.0)
Epoch 013/020 | Loss nan | Win/lose count 2.5/1.0 (1.5)
Epoch 014/020 | Loss nan | Win/lose count 4.0/6.0 (-2.0)
Epoch 015/020 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 016/020 | Loss nan | Win/lose count 2.0/3.0 (-1.0)
Epoch 017/020 | Loss nan | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [48]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('\nTest of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')


Test of the CNN
Win/lose count 17.0/1.0. Average score (16.0)
Win/lose count 12.5/1.0. Average score (13.75)
Win/lose count 19.0/0. Average score (15.5)
Win/lose count 10.5/2.0. Average score (13.75)
Win/lose count 7.0/2.0. Average score (12.0)
Win/lose count 15.5/2.0. Average score (12.25)
Win/lose count 17.0/2.0. Average score (12.642857142857142)
Win/lose count 12.0/3.0. Average score (12.1875)
Win/lose count 20.0/6.0. Average score (12.38888888888889)
Win/lose count 8.5/1.0. Average score (11.9)
Win/lose count 15.5/5.0. Average score (11.772727272727273)
Win/lose count 10.0/3.0. Average score (11.375)
Win/lose count 19.5/5.0. Average score (11.615384615384615)
Win/lose count 18.0/3.0. Average score (11.857142857142858)
Win/lose count 19.0/3.0. Average score (12.133333333333333)
Win/lose count 19.0/8.0. Average score (12.0625)
Win/lose count 14.5/4.0. Average score (11.970588235294118)
Win/lose count 9.0/2.0. Average score (11.694444444444445)
Win/lose count 10.0/0. Average score (

In [50]:
HTML(display_videos('cnn_test3.mp4'))

In [51]:
HTML(display_videos('fc_test10.mp4'))

#### Observations: 
we can obviously notice the fact that the model based on CNN performs distinctly better than the FC model. However, the agent doesn't explore other areas in the board and most of the time it sticks to the current board, going back and forth; this implies that we have a low exploration rate. 


#### Effect of temperature change :
We notice that we improve the final score, but it might be biased because we increase the probability of having cheese and we lower the probility of poisonious cells by doing this.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [52]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
             
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [53]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size,  epsilon = 1, memory_size=16000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')

Epoch 000/020 | Loss 0.0137 | Win/lose count 5.0/12.579999999999966 (-7.579999999999966)
Epoch 001/020 | Loss 0.0103 | Win/lose count 4.450000000000001/5.839999999999982 (-1.389999999999981)
Epoch 002/020 | Loss 0.0757 | Win/lose count 10.43/6.599999999999973 (3.8300000000000267)
Epoch 003/020 | Loss 0.1518 | Win/lose count 9.92/3.699999999999985 (6.220000000000015)
Epoch 004/020 | Loss 0.1982 | Win/lose count 14.810000000000004/1.6400000000000012 (13.170000000000003)
Epoch 005/020 | Loss 0.2044 | Win/lose count 10.320000000000002/1.7700000000000014 (8.55)
Epoch 006/020 | Loss 0.1337 | Win/lose count 7.350000000000002/1.8500000000000014 (5.500000000000001)
Epoch 007/020 | Loss 0.2264 | Win/lose count 10.780000000000003/2.7799999999999985 (8.000000000000004)
Epoch 008/020 | Loss 0.3012 | Win/lose count 21.18999999999999/2.529999999999989 (18.66)
Epoch 009/020 | Loss 0.6020 | Win/lose count 14.220000000000004/1.7000000000000013 (12.520000000000003)
Epoch 010/020 | Loss 0.2880 | Win/lose 

In [57]:
HTML(display_videos('cnn_train_explore14.mp4'))

In [55]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')


Win/lose count 13.230000000000004/1.7400000000000013. Average score (11.490000000000002)
Win/lose count 9.310000000000002/1.8200000000000014. Average score (9.490000000000002)
Win/lose count 5.880000000000002/2.8899999999999815. Average score (7.323333333333341)
Win/lose count 7.360000000000002/2.8599999999999826. Average score (6.617500000000011)
Win/lose count 6.870000000000002/1.8700000000000014. Average score (6.294000000000009)
Win/lose count 4.900000000000001/1.9000000000000015. Average score (5.745000000000008)
Win/lose count 13.240000000000004/2.739999999999985. Average score (6.424285714285723)
Win/lose count 8.830000000000002/1.8300000000000014. Average score (6.496250000000009)
Win/lose count 16.69/4.669999999999968. Average score (7.110000000000011)
Win/lose count 8.820000000000002/3.8099999999999627. Average score (6.900000000000015)
Win/lose count 16.660000000000004/2.669999999999986. Average score (7.544545454545468)
Win/lose count 7.350000000000002/2.8499999999999828. A

In [58]:
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***